<a href="https://colab.research.google.com/github/clabaudot/AI-ToDo/blob/main/Claris_VAS_AI_ToDo_prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI-ToDo **README**
My project is to create an AI assistant to manage a todo list. This is a project done in 6 weeks to practice AI capabilities

# 1. Overview of the AI ToDo
This project is to suggest a time in the week to execute a task from a todo list for the week. It doesn't contain any obvious activities like sleeping, eating, working, dressing, washing, commuting to work. It can contain additional tasks that the user would like to accomplish during the week.
The tasks can be simple, or complex, in case of a complex task the model will split it in subtasks. The model will recognize the input tasks from a list on generic tasks (the dataset).

* Step 1 : from a todo list generate the list of tasks with their characteristics.

* Step 2 : propose a time in the week to execute the task, during free time, linked to my calendar so there is no overlap with appointments. The model will also respect some of my criteria (not a morning person, working hours, ...).

* Step 3 : update the tasks list daily to mark the progress (done, partially done, reschedule next week, ...).

* Step 4 : send regular notifications during the week measuring the progress and motivating to complete all the tasks.

## Step 1
### Input Data Used
The input data is a list of text representing a todo list for the week.
example: ('bike ride', 'pay electricity bill', 'decorate house for christmas', 'clean the bathroom', 'call my friend Linda')

### Output Data
The output data is a JSON file containing the same list of tasks (and maybe subtasks) with an estimated duration, a type, a difficulty level, an indicator if it's inside or outside, an indicator if it requires travelling, a suggested time in the week to do it.

## Step 2
### Input Data Used
Use the list of tasks and subtasks with their characteristics obtained at step1. Json format

### Output Data
Proposed schedule of all the tasks spread over the week, based on my criteria.
Note: not linked to my calendar yet, but that's the plan

## Models Used
OpenAI GPT-4o-mini

## Evaluation Method
Not sure yet

# **2. Instructions**
[To be completed]
## Quick Start
## Setup
## Run


# **3. Input Data, Output Data, and Agent Code**

In [26]:
# Install necessary packages , just ONE TIME, comment after.
#!pip install --upgrade openai httpx==0.23.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Successfully uninstalled h11-0.14.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 1.0.7
    Uninstalling httpcore-1.0.7:
      Successfully uninstalled httpcore-1.0.7
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [1]:
# Import Libraries
import random, openai, json
import pandas as pd
from google.colab import userdata

In [2]:
# If you would like to use openai,
# please define the openai_key below otherwise leave as None
openai_key = userdata.get('openaikey')


In [3]:
# Get Input Data
# Example of input data. Format is a list of text
input_data = """
('bike ride', 'pay electricity bill', 'decorate house for christmas', 'clean the bathroom', 'call my friend Linda')
"""

In [4]:
# Get Output Data
# Example of output data. Format is a dictionary or json file
output_data = """
list of tasks (and maybe subtasks) with an estimated duration, a category, a difficulty level, an indicator if it's inside or outside, an indicator if it requires travelling, a suggested time in the week to do it.
"""

In [5]:
# Create my Agent
class ToDoAgent:
    def __init__(self):
        pass


    def predict_tasks_with_llm(self, task_list, api_key=None):
        """
        Generate list of tasks with characteristices using OpenAI's GPT model.
        """
        # Set OpenAI API key if provided
        if api_key:
            openai.api_key = api_key

        # Create a prompt to instruct OpenAI
        prompt = f"""
          Get a list of tasks with their characteristics based on the following list of strings

          {task_list}

          Each resulting task should be in JSON format with these fields:
		      - "task_ID": unique integer
		      - "task_name": text
          - "estimated_duration": a number in minutes
          - "category": a text defining a general category
          - "difficulty_level": a value within ("easy", "medium", "difficult")
          - "ind_outside": a boolean indicating if the task is done inside or outside
          - "ind_travel": a boolean indicating if the task requires travelling
          - "status": a value within ("not started", "done", "partially done", "reschedule"), initially this value is not started
          - "actual_duration": a number in minutes, initially this value is 0
          - "estimated_remaining_duration": a number in minutes or hours, initially this value is 0

		      For long or difficult tasks, create meaningful smaller subtasks.

          Return only a JSON array of the tasks.
          """
        client = openai.OpenAI(api_key=api_key)
        #client = openai.OpenAI()
        # Call OpenAI API to generate questions
        response = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            model="gpt-4o-mini",
        )
        # Parse and return the JSON response
        return response.choices[0].message.content

    def predict_timeslots_with_llm(self, tasks_subtasks, api_key=None):
        """
        Propose timeslots for my tasks during the week using OpenAI's GPT model.
        """
        # Set OpenAI API key if provided
        if api_key:
            openai.api_key = api_key

        # Create a prompt to instruct OpenAI
        prompt = f"""
          Propose some timeslots in my week to accomplish the following tasks:

          {tasks_subtasks}

          The result should be a list of tasks or subtasks with the ID, name, proposed day and time in the current week

		      The week starts on Monday.
          Avoid working time which is Monday to Friday from 9:00 amd to 5:00pm. Lunch time 12:00pm to 1:00pm can be used except Wednesdays
          Wednesday I work at the office so avoid the period of 1h commute before and after work.
          Avoid sleeping time from 11pm to 7am.
          Not a morning person so afternoon and evening are better choices.
          If the task is outdoor, plan for 1h to get at the location.
          Balance outdoor and indoor tasks over the week.
          Balance fun tasks and boring tasks over the week.
          Propose timeslots for subtasks and not the main task when there are subtasks.

          """
        client = openai.OpenAI(api_key=api_key)
        #client = openai.OpenAI()
        # Call OpenAI API to generate questions
        response = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            model="gpt-4o-mini",
        )
        # Parse and return the JSON response
        return response.choices[0].message.content


# 3. Run

In [6]:
# Get Output with my Agent
agent = ToDoAgent()


# Provide my todo list
todo_list = ['bike ride', 'pay electricity bill', 'decorate house for christmas', 'clean the bathroom', 'call my friend Linda']

# Get tasks predicted with duration and characteristics
generated_text = agent.predict_tasks_with_llm(task_list=todo_list, api_key=openai_key)

# parse output. This is response from chatgpt. JSON format
#if openai_key:
#  print(generated_text)

# parse output. This is the response from chatgpt in JSON format
if openai_key:
  # remove header for json result and any leading or trailing whitespace and
  # backticks from the response
  if generated_text.startswith("```json"):
    generated_text = generated_text[len("```json"):]
  if generated_text.endswith("```"):
    generated_text = generated_text[:-len("```")]
  json_tasks = generated_text.strip().strip("")

  # Convert the JSON string to a Python list of dictionaries
  tasks = json.loads(json_tasks)

  # Print as a table, use pandas dataframe
  # Create a pandas DataFrame from the list of dictionaries
  df_main_tasks = pd.DataFrame(tasks)

  # Display the DataFrame as a table
  display(df_main_tasks) # Use display for better formatting in Jupyter Notebook

  # If there are subtasks , print them in a different table
  subtasks = []
  for task in tasks:
    if 'subtasks' in task and task['subtasks']: # Check if subtasks exist for the task
      for subtask in task['subtasks']:
        subtask['parent_task_ID'] = task['task_ID'] # Add parent task ID to subtask
        subtasks.append(subtask)

  if subtasks:
    df_subtasks = pd.DataFrame(subtasks)
    display(df_subtasks) # Display subtasks as a table print("Subtasks:")
else:
  print(generated_text)

# Get tasks predicted with duration and characteristics
generated_schedule = agent.predict_timeslots_with_llm(tasks_subtasks=json_tasks, api_key=openai_key)

# parse output. This is response from chatgpt
if openai_key:
  print(generated_schedule)



,task_ID,task_name,estimated_duration,category,difficulty_level,ind_outside,ind_travel,status,actual_duration,estimated_remaining_duration,subtasks
0,1,bike ride,60,exercise,medium,True,True,not started,0,0,NaN
1,2,pay electricity bill,15,finance,easy,False,False,not started,0,0,NaN
2,3,decorate house for christmas,120,home improvement,medium,True,False,not started,0,0,"[{'task_ID': 3.1, 'task_name': 'buy christmas ..."
3,4,clean the bathroom,45,cleaning,medium,False,False,not started,0,0,NaN
4,5,call my friend Linda,15,communication,easy,False,False,not started,0,0,NaN


,task_ID,task_name,estimated_duration,category,difficulty_level,ind_outside,ind_travel,status,actual_duration,estimated_remaining_duration,parent_task_ID
0,3.1,buy christmas decorations,30,shopping,easy,True,True,not started,0,0,3
1,3.2,set up christmas tree,30,home improvement,medium,False,False,not started,0,0,3
2,3.3,hang christmas lights,30,home improvement,medium,True,False,not started,0,0,3


Based on your guidelines and constraints, here’s a proposed schedule for the tasks you've listed. I've considered the need to balance outdoor and indoor tasks, fun and boring tasks, and avoided working hours and commuting times. 

### Proposed Weekly Schedule:

#### Monday
- **5:00 PM - 6:00 PM**: Task 1 - **Bike Ride**

#### Tuesday
- **5:00 PM - 5:15 PM**: Task 2 - **Pay Electricity Bill**
- **5:30 PM - 6:00 PM**: Task 5 - **Call my friend Linda**

#### Wednesday
- **Evening free** (commuting and work obligations)

#### Thursday
- **5:00 PM - 6:00 PM**: Task 3.1 - **Buy Christmas Decorations** (1 hour travel + shopping)
- **6:30 PM - 7:00 PM**: Task 3.2 - **Set up Christmas Tree**

#### Friday
- **5:00 PM - 5:45 PM**: Task 4 - **Clean the Bathroom**
- **6:00 PM - 7:00 PM**: Task 3.3 - **Hang Christmas Lights**

#### Saturday
- **10:00 AM - 11:00 AM**: Task 1 - **Bike Ride**  
- **11:30 AM - 12:30 PM**: Task 3 - **Decorate House for Christmas** (Finish Setting Up Christmas decorations